In [ ]:
import argparse
import json
import random
from pathlib import Path

import pandas as pd


# ---------------------------
# НАСТРОЙКИ ПОД ВАШ ДАТАСЕТ
# ---------------------------

COLUMN_MAP = {
    "script_id": "script_id",
    "scene_id": "scene_id",
    "scene_text": "scene_text",      # текст сцены / блок сценария
    "location": "location",
    "time_of_day": "time_of_day",
    "characters": "characters",
    "extras": "extras",
    "props": "props",
    "fx": "fx",                      # спецэффекты
    # "notes": "notes",              # если есть — можно добавить
}

# Разделитель списков в ячейках (персонажи, реквизит и т.п.)
LIST_SEPARATOR = ";"


def load_table(path: Path) -> pd.DataFrame:
    """Загрузка CSV или Excel в pandas."""
    if path.suffix.lower() in [".xlsx", ".xls"]:
        df = pd.read_excel(path)
    else:
        df = pd.read_csv(path)
    return df


def normalize_list_cell(value):
    """
    Преобразуем строку вроде "МИША; АЛИНА;  " в список ["МИША", "АЛИНА"].
    Если NaN/пусто — возвращаем пустой список.
    """
    if pd.isna(value):
        return []
    if isinstance(value, list):
        # Уже список
        return [str(x).strip() for x in value if str(x).strip()]
    text = str(value)
    parts = [p.strip() for p in text.split(LIST_SEPARATOR)]
    return [p for p in parts if p]


def normalize_time_of_day(value):
    """
    Нормализация времени суток к фиксированному набору.
    При необходимости расширьте/измените маппинг.
    """
    if pd.isna(value):
        return None
    text = str(value).strip().lower()

    mapping = {
        "день": "day",
        "днём": "day",
        "утро": "morning",
        "ночь": "night",
        "ночью": "night",
        "вечер": "evening",
        "вечером": "evening",
        "day": "day",
        "night": "night",
        "morning": "morning",
        "evening": "evening",
    }

    # если совпадает напрямую
    if text in mapping:
        return mapping[text]

    # простые эвристики
    if "ноч" in text:
        return "night"
    if "утр" in text:
        return "morning"
    if "вечер" in text:
        return "evening"
    if "день" in text or "day" in text:
        return "day"

    return None  # если не смогли классифицировать


def normalize_location(value):
    """Простейшая нормализация локации (обрезка пробелов, приведение пробелов)."""
    if pd.isna(value):
        return None
    text = " ".join(str(value).split())
    return text if text else None


def filter_good_scenes(df: pd.DataFrame, min_text_len: int = 50):
    """
    Отбираем сцены, которые подойдут в золотой стандарт:
    - есть текст
    - есть локация и/или время суток
    - длина текста не слишком маленькая
    """
    df = df.copy()

    # нормализуем базовые поля
    df["scene_text"] = df[COLUMN_MAP["scene_text"]].astype(str).str.strip()

    df["location_norm"] = df[COLUMN_MAP["location"]].map(normalize_location)
    df["time_of_day_norm"] = df[COLUMN_MAP["time_of_day"]].map(normalize_time_of_day)

    # минимальная длина текста
    df = df[df["scene_text"].str.len() >= min_text_len]

    # хотя бы что-то одно: локация или время суток
    df = df[(df["location_norm"].notna()) | (df["time_of_day_norm"].notna())]

    return df


def row_to_example(row) -> dict:
    """
    Преобразование строки DataFrame в JSON-пример для обучения/оценки.
    """
    example = {
        "script_id": str(row[COLUMN_MAP["script_id"]]),
        "scene_id": str(row[COLUMN_MAP["scene_id"]]),
        "text": str(row["scene_text"]),
        "labels": {
            "location": row["location_norm"],
            "time_of_day": row["time_of_day_norm"],
            "characters": normalize_list_cell(row.get(COLUMN_MAP["characters"], None)),
            "extras": normalize_list_cell(row.get(COLUMN_MAP["extras"], None)),
            "props": normalize_list_cell(row.get(COLUMN_MAP["props"], None)),
            "fx": normalize_list_cell(row.get(COLUMN_MAP["fx"], None)),
        },
    }

    # Можно добавить notes, если есть
    # if "notes" in COLUMN_MAP:
    #     example["labels"]["notes"] = str(row.get(COLUMN_MAP["notes"], ""))

    return example


def save_jsonl(examples, path: Path):
    """Сохраняем список dict в JSONL."""
    with path.open("w", encoding="utf-8") as f:
        for ex in examples:
            f.write(json.dumps(ex, ensure_ascii=False) + "\n")


def split_train_dev_test(examples, train_ratio=0.8, dev_ratio=0.1, seed=42):
    """
    Разбиваем примеры на train / dev / test.
    По умолчанию: 80% / 10% / 10%.
    """
    random.Random(seed).shuffle(examples)
    n = len(examples)
    n_train = int(n * train_ratio)
    n_dev = int(n * dev_ratio)
    n_test = n - n_train - n_dev

    train = examples[:n_train]
    dev = examples[n_train:n_train + n_dev]
    test = examples[n_train + n_dev:]

    return train, dev, test


def main():
    parser = argparse.ArgumentParser(
        description="Выделение золотого стандарта из табличного датасета сценариев."
    )
    parser.add_argument(
        "--input",
        type=str,
        required=True,
        help="Путь к входному CSV/Excel с таблицей сцен.",
    )
    parser.add_argument(
        "--output_dir",
        type=str,
        required=True,
        help="Папка для сохранения JSONL (train/dev/test).",
    )
    parser.add_argument(
        "--min_text_len",
        type=int,
        default=50,
        help="Минимальная длина текста сцены для попадания в золотой стандарт.",
    )
    parser.add_argument(
        "--max_examples",
        type=int,
        default=0,
        help="Максимальное количество примеров (0 = использовать все подходящие).",
    )

    args = parser.parse_args()

    input_path = Path(args.input)
    output_dir = Path(args.output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    print(f"Загружаем таблицу из: {input_path}")
    df = load_table(input_path)

    print("Фильтруем сцены для золотого стандарта...")
    df_good = filter_good_scenes(df, min_text_len=args.min_text_len)
    print(f"Подходящих сцен: {len(df_good)}")

    examples = [row_to_example(row) for _, row in df_good.iterrows()]

    if args.max_examples > 0 and len(examples) > args.max_examples:
        print(f"Ограничиваем число примеров до {args.max_examples}")
        examples = random.sample(examples, args.max_examples)

    train, dev, test = split_train_dev_test(examples)

    print(f"train: {len(train)}, dev: {len(dev)}, test: {len(test)}")

    save_jsonl(train, output_dir / "gold_train.jsonl")
    save_jsonl(dev, output_dir / "gold_dev.jsonl")
    save_jsonl(test, output_dir / "gold_test.jsonl")

    print("Готово. Файлы сохранены:")
    print(f"- {output_dir / 'gold_train.jsonl'}")
    print(f"- {output_dir / 'gold_dev.jsonl'}")
    print(f"- {output_dir / 'gold_test.jsonl'}")


if __name__ == "__main__":
    main()


usage: colab_kernel_launcher.py [-h] --input INPUT --output_dir OUTPUT_DIR
                                [--min_text_len MIN_TEXT_LEN]
                                [--max_examples MAX_EXAMPLES]
colab_kernel_launcher.py: error: the following arguments are required: --input, --output_dir
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/lib/python3.12/argparse.py", line 1943, in _parse_known_args2
    namespace, args = self._parse_known_args(args, namespace, intermixed)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/argparse.py", line 2230, in _parse_known_args
    raise ArgumentError(None, _('the following arguments are required: %s') %
argparse.ArgumentError: the following arguments are required: --input, --output_dir

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-3735302977.py", line 239, in <cell line: 0>
    main()
  File "/tmp/ipython-input-3735302977.py", line 205, in main
    args = parser.parse_args()
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/argparse

TypeError: object of type 'NoneType' has no len()

юпд после кода ревью

In [ ]:
import argparse
import json
import random
from collections import defaultdict
from pathlib import Path
import ast

import pandas as pd


# ---------------------------
# НАСТРОЙКИ ПОД ВАШ ДАТАСЕТ
# ---------------------------

COLUMN_MAP = {
    "script_id": "script_id",
    "scene_id": "scene_id",
    "scene_text": "scene_text",      # текст сцены / блок сценария
    "location": "location",
    "time_of_day": "time_of_day",
    "characters": "characters",
    "extras": "extras",
    "props": "props",
    "fx": "fx",                      # спецэффекты
    # "notes": "notes",              # если есть — можно добавить
}

# Разделитель списков в ячейках (персонажи, реквизит и т.п.)
LIST_SEPARATOR = ";"


# ---------------------------
# ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ
# ---------------------------

def load_table(path: Path) -> pd.DataFrame:
    """Загрузка CSV или Excel в pandas."""
    if path.suffix.lower() in [".xlsx", ".xls"]:
        df = pd.read_excel(path)
    else:
        df = pd.read_csv(path)
    return df


def validate_columns(df: pd.DataFrame):
    """Проверяем наличие обязательных колонок в датасете."""
    required_columns = [
        COLUMN_MAP["script_id"],
        COLUMN_MAP["scene_id"],
        COLUMN_MAP["scene_text"],
        COLUMN_MAP["location"],
        COLUMN_MAP["time_of_day"],
    ]
    missing = [c for c in required_columns if c not in df.columns]
    if missing:
        raise ValueError(f"Отсутствуют обязательные колонки в датасете: {missing}")


def safe_str(v):
    """Безопасное преобразование в строку (NaN → None)."""
    if pd.isna(v):
        return None
    return str(v)


def normalize_list_cell(value):
    """
    Преобразуем содержимое ячейки в список строк.
    Поддерживаем:
    - NaN → []
    - уже список → нормализуем элементы
    - JSON-список в строке → парсим
    - строку с разделителем LIST_SEPARATOR → сплитим
    """
    if pd.isna(value):
        return []
    if isinstance(value, list):
        return [str(x).strip() for x in value if str(x).strip()]

    text = str(value).strip()
    if not text:
        return []

    # Пытаемся распарсить JSON-подобный список: ["МИША", "АЛИНА"]
    if text.startswith("[") and text.endswith("]"):
        try:
            arr = ast.literal_eval(text)
            if isinstance(arr, list):
                return [str(x).strip() for x in arr if str(x).strip()]
        except Exception:
            pass

    parts = [p.strip() for p in text.split(LIST_SEPARATOR)]
    return [p for p in parts if p]


def normalize_time_of_day(value):
    """
    Нормализация времени суток к фиксированному набору.
    При необходимости расширьте/измените маппинг.
    """
    if pd.isna(value):
        return None
    text = str(value).strip().lower()

    mapping = {
        "день": "day",
        "днём": "day",
        "днем": "day",
        "утро": "morning",
        "ночь": "night",
        "ночью": "night",
        "вечер": "evening",
        "вечером": "evening",
        "day": "day",
        "night": "night",
        "morning": "morning",
        "evening": "evening",
    }

    if text in mapping:
        return mapping[text]

    # простые эвристики, если значение не нормализовано
    if "ноч" in text:
        return "night"
    if "утр" in text:
        return "morning"
    if "вечер" in text:
        return "evening"
    if "день" in text or "day" in text:
        return "day"

    return None  # если не смогли классифицировать


def normalize_location(value):
    """Простейшая нормализация локации (трим + схлопывание пробелов)."""
    if pd.isna(value):
        return None
    text = " ".join(str(value).split())
    return text if text else None


def filter_good_scenes(df: pd.DataFrame, min_text_len: int = 50):
    """
    Отбираем сцены, которые подойдут в золотой стандарт:
    - есть текст
    - есть локация и/или время суток
    - длина текста не слишком маленькая
    """
    df = df.copy()
    total = len(df)
    print(f"Всего строк в исходном датасете: {total}")

    # нормализуем базовые поля
    df["scene_text"] = df[COLUMN_MAP["scene_text"]].astype(str).str.strip()
    df["location_norm"] = df[COLUMN_MAP["location"]].map(normalize_location)
    df["time_of_day_norm"] = df[COLUMN_MAP["time_of_day"]].map(normalize_time_of_day)

    # минимальная длина текста
    before_len = len(df)
    df = df[df["scene_text"].str.len() >= min_text_len]
    after_len = len(df)
    print(f"Фильтр по длине текста (>= {min_text_len}): оставлено {after_len}/{before_len}")

    # хотя бы что-то одно: локация или время суток
    before_loc = len(df)
    df = df[(df["location_norm"].notna()) | (df["time_of_day_norm"].notna())]
    after_loc = len(df)
    print(f"Фильтр по локации/времени суток: оставлено {after_loc}/{before_loc}")

    return df


def row_to_example(row) -> dict:
    """
    Преобразование строки DataFrame в JSON-пример для обучения/оценки.
    """
    example = {
        "script_id": safe_str(row[COLUMN_MAP["script_id"]]),
        "scene_id": safe_str(row[COLUMN_MAP["scene_id"]]),
        "text": str(row["scene_text"]),
        "labels": {
            "location": row["location_norm"],
            "time_of_day": row["time_of_day_norm"],
            "characters": normalize_list_cell(row.get(COLUMN_MAP["characters"], None)),
            "extras": normalize_list_cell(row.get(COLUMN_MAP["extras"], None)),
            "props": normalize_list_cell(row.get(COLUMN_MAP["props"], None)),
            "fx": normalize_list_cell(row.get(COLUMN_MAP["fx"], None)),
        },
    }

    # Можно добавить notes, если есть
    # if "notes" in COLUMN_MAP:
    #     example["labels"]["notes"] = safe_str(row.get(COLUMN_MAP["notes"], ""))

    return example


def save_jsonl(examples, path: Path):
    """Сохраняем список dict в JSONL."""
    with path.open("w", encoding="utf-8") as f:
        for ex in examples:
            f.write(json.dumps(ex, ensure_ascii=False) + "\n")


def split_train_dev_test_by_script(
    examples,
    train_ratio=0.8,
    dev_ratio=0.1,
    seed=42,
):
    """
    Разбиваем примеры на train / dev / test по script_id,
    чтобы сцены одного сценария не попадали в разные сплиты.
    """
    random.seed(seed)

    by_script = defaultdict(list)
    for ex in examples:
        sid = ex["script_id"]
        by_script[sid].append(ex)

    scripts = list(by_script.keys())
    random.shuffle(scripts)

    n_scripts = len(scripts)
    n_train = int(n_scripts * train_ratio)
    n_dev = int(n_scripts * dev_ratio)
    n_test = n_scripts - n_train - n_dev

    train_scripts = set(scripts[:n_train])
    dev_scripts = set(scripts[n_train:n_train + n_dev])
    test_scripts = set(scripts[n_train + n_dev:])

    train, dev, test = [], [], []
    for sid, scenes in by_script.items():
        if sid in train_scripts:
            train.extend(scenes)
        elif sid in dev_scripts:
            dev.extend(scenes)
        else:
            test.extend(scenes)

    print(f"Сценариев: всего={n_scripts}, train={len(train_scripts)}, dev={len(dev_scripts)}, test={len(test_scripts)}")
    print(f"Примеров: train={len(train)}, dev={len(dev)}, test={len(test)}")

    return train, dev, test


# ---------------------------
# MAIN
# ---------------------------

def main():
    parser = argparse.ArgumentParser(
        description="Выделение золотого стандарта из табличного датасета сценариев."
    )
    parser.add_argument(
        "--input",
        type=str,
        required=True,
        help="Путь к входному CSV/Excel с таблицей сцен.",
    )
    parser.add_argument(
        "--output_dir",
        type=str,
        required=True,
        help="Папка для сохранения JSONL (train/dev/test).",
    )
    parser.add_argument(
        "--min_text_len",
        type=int,
        default=50,
        help="Минимальная длина текста сцены для попадания в золотой стандарт.",
    )
    parser.add_argument(
        "--max_examples",
        type=int,
        default=0,
        help="Максимальное количество примеров (0 = использовать все подходящие).",
    )

    args = parser.parse_args()

    input_path = Path(args.input)
    output_dir = Path(args.output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    print(f"Загружаем таблицу из: {input_path}")
    df = load_table(input_path)

    print("Проверяем наличие обязательных колонок...")
    validate_columns(df)

    print("Фильтруем сцены для золотого стандарта...")
    df_good = filter_good_scenes(df, min_text_len=args.min_text_len)
    print(f"Подходящих сцен: {len(df_good)}")

    print("Преобразуем строки в обучающие примеры...")
    examples = [row_to_example(row) for _, row in df_good.iterrows()]

    if args.max_examples > 0 and len(examples) > args.max_examples:
        print(f"Ограничиваем число примеров до {args.max_examples}")
        examples = random.sample(examples, args.max_examples)

    print("Разбиваем на train/dev/test по script_id...")
    train, dev, test = split_train_dev_test_by_script(examples)

    print("Сохраняем JSONL-файлы...")
    save_jsonl(train, output_dir / "gold_train.jsonl")
    save_jsonl(dev, output_dir / "gold_dev.jsonl")
    save_jsonl(test, output_dir / "gold_test.jsonl")

    print("Готово. Файлы сохранены:")
    print(f"- {output_dir / 'gold_train.jsonl'}")
    print(f"- {output_dir / 'gold_dev.jsonl'}")
    print(f"- {output_dir / 'gold_test.jsonl'}")


if __name__ == "__main__":
    main()
